In [32]:
#pyodbc  open db connectivite

In [188]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [189]:
import pyodbc

In [190]:
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client 11.0',
 'ODBC Driver 17 for SQL Server',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 13 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)',
 'PostgreSQL ANSI(x64)',
 'PostgreSQL Unicode(x64)']

In [191]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try:
        conn = pyodbc.connect(DRIVER = driver, SERVER= server, DATABASE = database, UID= user, PWD= password)
        print("Successfull")
    except Error as err:
        print(f"Error:'{err}'")
    return conn

In [192]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost' #şu anda server her birimizin kendi makinesi
database = 'master'#yeni bir db oluşturacağız
user = 'sa'
password = 'Th44Tr00' # şifre önemli
conn = create_server_connection(driver, server, database, user, password)

Successfull


# Windows authentication

In [197]:
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=master; TRUSTED_CONNECTION=yes;"

In [198]:
conn = pyodbc.connect(conn_string);

In [199]:
crs=conn.cursor()

In [200]:
conn.autocommit = True

In [91]:
conn.commit()

In [92]:
create_database_query='Create database test_1'

In [201]:
def create_database (conn, create_database_query):
    crs = conn.cursor()
    try:
        crs.execute(create_database_query)
        print("Database is Created Succesfully")
    except Error as err:
        print(f"Error:'{err}'")

In [202]:
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [203]:
create_database(conn,create_database_query)

NameError: name 'Error' is not defined

In [204]:
query ='USE Test_1'
execute_query(conn,query)

Query Succeessful!


In [206]:
query = 'CREATE TABLE TestA (\
           ID INT IDENTITY (1,1) NOT NULL, \
           FirstName VARCHAR(255) NOT NULL,\
           LastName VARCHAR(255) NOT NULL,\
           PRIMARY KEY(ID))'
execute_query(conn, query)

NameError: name 'Error' is not defined

#  İnsert

In [207]:
crs= conn.cursor()
crs.execute("INSERT TestA (FirstName, LastName) VALUES ('Bob', 'Marley')")
#conn.commit()

In [208]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Bruce', 'Lee')

crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Tom', 'Hanks')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Bart', 'Simpson')

crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Tom', 'Cat')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Jerry', 'Mouse')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Owen', 'William')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Stefan', 'Müller')

# Delete

In [209]:
crs.execute("DELETE FROM TestA WHERE FirstName = 'Jerry'")

In [210]:
crs.execute("DELETE FROM TestA WHERE FirstName = 'Stefan'")

# Rowcout

In [211]:
crs.rowcount  # etkilne satır sayısı

1

In [212]:
print(crs.rowcount,"row(s) deleted")

1 row(s) deleted


# Read data

# fetchone()

In [213]:
crs.execute('SELECT FirstName, LastName FROM TestA')
row = crs.fetchone() # tablodaki ilk kaydı getirir
row

('Bob', 'Marley')

In [214]:
row = crs.fetchone() # tablodaki ilk kaydı getirir
row

('Bruce', 'Lee')

In [215]:
row = crs.fetchone() # tablodaki ilk kaydı getirir
row

('Tom', 'Hanks')

In [216]:
row.FirstName  # row dan first name mi aldık

'Tom'

# fetchall()

In [217]:
crs.execute("""SELECT ID, FirstName, LastName
                FROM TestA""")
rows = crs.fetchall()
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (7, 'Owen', 'William'),
 (9, 'Bob', 'Marley'),
 (10, 'Bruce', 'Lee'),
 (11, 'Tom', 'Hanks'),
 (12, 'Bart', 'Simpson'),
 (13, 'Tom', 'Cat'),
 (15, 'Owen', 'William')]

In [218]:
for row in rows:
    print(row.ID,row.FirstName,row.LastName)

1 Bob Marley
2 Bruce Lee
3 Tom Hanks
4 Bart Simpson
5 Tom Cat
7 Owen William
9 Bob Marley
10 Bruce Lee
11 Tom Hanks
12 Bart Simpson
13 Tom Cat
15 Owen William


In [219]:
type(rows)

list

In [220]:
crs.execute("""SELECT ID, FirstName, LastName
               FROM TestA
               WHERE FirstName = ? and LastName = ?""", ['Owen', 'William'])
rows = crs.fetchall()
rows   # ? ile filtre uyguladık

[(7, 'Owen', 'William'), (15, 'Owen', 'William')]

# fetchmany()

In [221]:
crs.execute("""SELECT ID, FirstName, LastName
                FROM TestA""")
rows = crs.fetchmany(3)  # kaç değer varsa getirdik
rows

[(1, 'Bob', 'Marley'), (2, 'Bruce', 'Lee'), (3, 'Tom', 'Hanks')]

# fetchval()

In [222]:
crs.execute("SELECT FirstName FROM TestA").fetchval()

'Bob'

In [223]:
crs.execute("SELECT MAX(ID) FROM TestA").fetchval()

15

# PANDAS Methods veri çekip df yapacağız

In [224]:
import pandas as pd

In [142]:
def read_query (conn, query): #connection nesnesi yukarıda tanımlanmıştı.
    crs = conn. cursor()
    results = None
    crs.execute(query)
    results = crs.fetchall()
    return results

In [143]:
query = """SELECT ID, FirstName, LastName
               FROM TestA"""

In [145]:
result = read_query (conn, query)
result

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (7, 'Owen', 'William')]

In [146]:
df= pd.DataFrame(result)

In [147]:
df

,0
0,"[1, Bob, Marley]"
1,"[2, Bruce, Lee]"
2,"[3, Tom, Hanks]"
3,"[4, Bart, Simpson]"
4,"[5, Tom, Cat]"
5,"[7, Owen, William]"


In [148]:
from_db = []
for result in result:
    result = list(result)
    from_db.append(result)

In [149]:
columns = ['ID','FirstName', 'LastName']
df = pd.DataFrame(from_db, columns = columns)
df

,ID,FirstName,LastName
0,1,Bob,Marley
1,2,Bruce,Lee
2,3,Tom,Hanks
3,4,Bart,Simpson
4,5,Tom,Cat
5,7,Owen,William


In [151]:
qery='USE BikeStores'
execute_query(conn,query)

Query Succeessful!


In [152]:
query = 'USE BikeStores'
execute_query(conn,query)

crs.execute("SELECT * FROM production.brands").fetchall()

Query Succeessful!


[(1, 'Electra'),
 (2, 'Haro'),
 (3, 'Heller'),
 (4, 'Pure Cycles'),
 (5, 'Ritchey'),
 (6, 'Strider'),
 (7, 'Sun Bicycles'),
 (8, 'Surly'),
 (9, 'Trek')]

In [153]:
pd.read_sql("SELECT * FROM production.brands", con = conn)

,brand_id,brand_name
0,1,Electra
1,2,Haro
2,3,Heller
3,4,Pure Cycles
4,5,Ritchey
5,6,Strider
6,7,Sun Bicycles
7,8,Surly
8,9,Trek


In [156]:
df_prod = pd.read_sql("SELECT * FROM production.products", con = conn)
df_prod.head(10)

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,Trek 820 - 2016,9,6,2016,379.99
1,2,Ritchey Timberwolf Frameset - 2016,5,6,2016,749.99
2,3,Surly Wednesday Frameset - 2016,8,6,2016,999.99
3,4,Trek Fuel EX 8 29 - 2016,9,6,2016,2899.99
4,5,Heller Shagamaw Frame - 2016,3,6,2016,1320.99
5,6,Surly Ice Cream Truck Frameset - 2016,8,6,2016,469.99
6,7,Trek Slash 8 27.5 - 2016,9,6,2016,3999.99
7,8,Trek Remedy 29 Carbon Frameset - 2016,9,6,2016,1799.99
8,9,Trek Conduit+ - 2016,9,5,2016,2999.99
9,10,Surly Straggler - 2016,8,4,2016,1549.00


# DF to Database()

In [177]:
import sqlalchemy
import urllib

In [178]:
crs.execute('USE Test_1')

In [182]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost' #şu anda server her birimizin kendi makinesi
database = 'Test_1'#yeni bir db oluşturacağız
user = 'sa'
password = 'Th44Tr00'

In [183]:
params = 'DRIVER='+driver + ';SERVER=' + server +';PORT=1433;DATABASE='+ database + ';UID=' + user + ';PWD=' + password

In [184]:
db_params = urllib.parse.quote_plus (params)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

In [185]:
df_prod = pd.read_sql("SELECT * FROM production.products where product_id <10", con = conn)
df_prod.head(10)

DatabaseError: Execution failed on sql 'SELECT * FROM production.products where product_id <10': ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'production.products'. (208) (SQLExecDirectW)")

In [186]:
df_prod.to_sql("products_bikes", engine, index = False, if_exists = "append", schema = "dbo")

In [187]:
crs.execute('USE Test_1')

driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost' #şu anda server her birimizin kendi makinesi
database = 'Test_1'#yeni bir db oluşturacağız
user = 'sa'
password = '---'

params = 'DRIVER='+driver + ';SERVER=' + server +';PORT=1433;DATABASE='+ database + ';UID=' + user + ';PWD=' + password

db_params = urllib.parse.quote_plus (params)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

df_prod = pd.read_sql("SELECT * FROM production.products where product_id <10", con = conn)
df_prod.head(10)

df_prod.to_sql("products_bikes", engine, index = False, if_exists = "append", schema = "dbo")

DatabaseError: Execution failed on sql 'SELECT * FROM production.products where product_id <10': ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'production.products'. (208) (SQLExecDirectW)")